##### 1. 증권사 리포트 수집
- Selenium을 활용한 추천

In [ ]:
# 수집할 종목 783개 Load
import pandas as pd

kospi_ticker_df= pd.read_csv('KOSPI_ticker.csv', encoding='EUC-KR', index_col=0)
kospi_ticker_df['종목코드'] = kospi_ticker_df['종목코드'].map(lambda x : '%06d'%x)
kospi_ticker_list = list(kospi_ticker_df['종목코드'])

In [ ]:
# Selenium을 활용한 수집
from selenium import webdriver 
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from tqdm import tqdm
import time


chrome_path = './chromedriver'
driver = webdriver.Chrome(chrome_path)

for stock_code in tqdm(kospi_ticker_list[-110:]):
    url = f'http://consensus.hankyung.com/apps.analysis/analysis.list?sdate=2020-06-13&edate=2021-06-13&now_page=1&search_value=&report_type=CO&pagenum=20&search_text={stock_code}&business_code='

    driver.get(url)
    html = driver.page_source                
    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find_all("table")[0]
    table = "<table>" + str(table) + "</table>" #pandas needs the table tag to recognize a table
    tempDataFrame = pd.read_html(table)[0]
    tempDataFrame.drop(['기업정보','차트'], axis=1, inplace=True)
    
    
    #pdf 첨부파일
    pdf_url_list = []
    pdf_url = "http://consensus.hankyung.com/"
    
    try:
        for i in np.arange(2, tempDataFrame.shape[0]*3, 3):
            pdf_url += soup.find_all(attrs={'class' : "dv_input"})[i].find_all('a')[0]['href']
            pdf_url_list.append(pdf_url)

        tempDataFrame.loc[:, '첨부파일'] = pdf_url_list

    except:
        pass
    
    #save
    report_json = tempDataFrame.to_json(orient="records", force_ascii=False)
    with open(f'report-crawling/report_{stock_code}.json', 'w', encoding='UTF-8-sig') as file:
        file.write(json.dumps(report_json, ensure_ascii=False))   
    
    time.sleep(0.1)
    
    display(tempDataFrame.tail(1))
    

In [ ]:
import pandas as pd
import json
import ast
from tqdm.notebook import tqdm

total_report_df = pd.DataFrame()

for stock_code in tqdm(kospi_ticker_list):
    with open(f'report-crawling/report_{stock_code}.json', 'r', encoding='UTF-8-sig') as file:
        json_load = json.load(file)

        stock_report_df = pd.DataFrame(ast.literal_eval(json_load))
        stock_report_df.loc[:, '종목코드'] = stock_code


        
        stock_report_df = stock_report_df[['종목코드','작성일','제목','적정가격',
                                           '투자의견','작성자','제공출처','첨부파일']]
        
        stock_report_df.rename(columns={'종목코드':'code',
                                        '작성일':'date',
                                        '제목':'title',
                                        '적정가격':'target',
                                        '투자의견':'opinion',
                                        '작성자':'writer',
                                        '제공출처':'from',
                                        '첨부파일':'link'}, inplace=True)  

        total_report_df = total_report_df.append(stock_report_df, ignore_index=True)
    

In [ ]:
total_report_df

In [ ]:
# json 저장

report_all_json = total_report_df.to_json(orient="records", force_ascii=False)

with open(f'report-crawling/report_all_english.json', 'w', encoding='UTF-8-sig') as file:
    file.write(json.dumps(report_all_json, ensure_ascii=False))   

##### 2. 기업 report가공 json Parsing

In [1]:
import json
import pandas as pd

report_all_df = pd.read_csv('report-crawling/report_all.csv')
report_all_df['첨부파일'] = report_all_df['첨부파일'].map(lambda x : x.replace('\/', '/'))
report_all_df['종목코드'] = report_all_df['종목코드'].map(lambda x : '%06d'%x)
report_all_df = report_all_df[~report_all_df.duplicated(subset=['종목코드','제목'])].reset_index(drop=True)

report_all_df.rename(columns={'종목코드':'code',' 작성일':'date', '제목':'title', '적정가격':'target', '작성일':'date',
                             '투자의견':'opinion', '작성자':'writer', '제공출처':'from', '첨부파일':'link'}, inplace=True)
report_all_df.to_json('report-all.json', force_ascii=False)

In [2]:
with open('report-all.json', 'r', encoding='utf-8-sig') as f:
    stock_report_json = json.load(f)
    
stock_report_df = pd.DataFrame(stock_report_json)
stock_report_df.tail(10)

,code,date,title,target,opinion,writer,from,link
4688,375500,2021-04-26,DL이앤씨(375500) 하반기로 갈수록 좋... DL이앤씨(375500) 하반기로...,160000,Buy,송유림,한화투자증권,http://consensus.hankyung.com//apps.analysis/a...
4689,375500,2021-04-13,"DL이앤씨(375500)저평가 상태, 모멘텀... DL이앤씨(375500)저평가 상...",165000,Buy,윤승현,하나금융투자,http://consensus.hankyung.com//apps.analysis/a...
4690,375500,2021-04-13,DL이앤씨(375500)갈수록 좋아지는 그... DL이앤씨(375500)갈수록 좋아...,0,Not Rated,신서정,SK증권,http://consensus.hankyung.com//apps.analysis/a...
4691,375500,2021-02-26,DL이앤씨(375500) 중기 전략 및 주주... DL이앤씨(375500) 중기 전...,0,Not Rated,김기룡,유안타증권,http://consensus.hankyung.com//apps.analysis/a...
4692,375500,2021-02-01,DL이앤씨(375500)싸게 사는 건 진리 DL이앤씨(375500)싸게 사는 건 진...,160000,Buy,"송유림,이재연",한화투자증권,http://consensus.hankyung.com//apps.analysis/a...
4693,375500,2021-01-29,DL이앤씨(375500)4Q20 Review: 밸류에... DL이앤씨(375500)...,0,nr,김세련,이베스트증권,http://consensus.hankyung.com//apps.analysis/a...
4694,378850,결과가 없습니다.,결과가 없습니다.,결과가 없습니다.,결과가 없습니다.,결과가 없습니다.,결과가 없습니다.,결과가 없습니다.
4695,383220,2021-05-28,F&F(383220)매수하지 않을 이유가 없... F&F(383220)매수하지 않을...,600000,Buy,하누리,메리츠증권,http://consensus.hankyung.com//apps.analysis/a...
4696,383220,2021-05-24,F&F(383220)잠시 쉬는 사이 더욱 강해... F&F(383220)잠시 쉬는 ...,550000,Buy,"손효주, 조은아",한화투자증권,http://consensus.hankyung.com//apps.analysis/a...
4697,383220,2021-05-24,F&F(383220)기다리는 조정은 없다 F&F(383220)기다리는 조정은 없다 ...,600000,Buy,하누리,메리츠증권,http://consensus.hankyung.com//apps.analysis/a...
